This file contains the code used for implementing the classifer in paper Comparison of Discrimination Methods for the Classification of Tumors Using Gene Expression data. This notebook could be pretty slow as a lot of function implemented are not time efficient enough in R (will try to optimize it if possible in the future).

**Data Set**: Merge the train and test data in the original paper and split the merged data to form train and test during learning.
    - Merge Data: 72 samples(47 ALL, 25 AML, 62 bone marrow, 10 peripheral blood samples)
    - Predictors: 7129 gene expression levels represent 6817 genes.
    
**Main Purpose**: LDA, QDA, Decision Tree, NN.

## Reproduce using R

### Step 1 Load and preprocess data.

In [1]:
# The implementation below is very similar to the one in paper1 notehbook and details chould be checked there.

## The code below is commented out since it is unnecessary and time-consuming to run it everytime. Run it if needed.
#options(repos='http://cran.rstudio.com/') 
#source("http://bioconductor.org/biocLite.R")
#biocLite("golubEsets")
#install.packages("tree")
#install.packages("fastAdaboost")
#install.packages("sparsediscrim", dependencies = T)
suppressMessages(library(sparsediscrim))
suppressMessages(library(tree))
suppressMessages(library(golubEsets))
suppressMessages(library(fastAdaboost))

# load data from golubEsets
data(Golub_Merge)
golub_merge_p = t(exprs(Golub_Merge))
golub_merge_r =pData(Golub_Merge)[, "ALL.AML"]
golub_merge_l = ifelse(golub_merge_r == "AML", 1, 0)

#show summary
dim(golub_merge_p) 
table(golub_merge_r)

# Thresholding
golub_merge_pp = golub_merge_p
golub_merge_pp[golub_merge_pp<100] = 100
golub_merge_pp[golub_merge_pp>16000] = 16000

# Filtering
golub_filter = function(x, r = 5, d=500){
    minval = min(x)
    maxval = max(x)
    (maxval/minval>r)&&(maxval-minval>d)
}
merge_index = apply(golub_merge_pp, 2, golub_filter)
golub_merge_index = (1:7129)[merge_index]
golub_merge_pp = golub_merge_pp[, golub_merge_index]

# Base 10 logarithmic transformation
golub_merge_p_trans = log10(golub_merge_pp)

#show summary again
dim(golub_merge_p_trans)
table(golub_merge_r)

# Further standardization to mean 0 variance 1.
scale_golub_merge = scale(golub_merge_p_trans)
set.seed(201703)

[1]   72 7129

golub_merge_r
ALL AML 
 47  25 

[1]   72 3571

golub_merge_r
ALL AML 
 47  25 

As we see in the result above, the dataset is indeed reduce to $72 \times 3571$ and this is also the origin of the $72 \times 3571$ dataset available online.

### Step 2 Build Classification

In [2]:
# Settings
p = 40 # number of genes for FLDA
B = 50 # Aggregation predictors
N = 200 # repeat classification N times
d = c(0.05, 0.1,0.25, 0.5, 0.75, 1) # CPD parameter

In [3]:
# Split train test
mysplit = function(n){
    sample(1:n, floor(n/3))
}
# calculate BW(the ratio of between-group to within group sums of squares)
BW = function(predictor, response){
    overall = colMeans(predictor)
    ALL_mean = apply(predictor, 2, function(x) mean(x[response == "ALL"]))
    AML_mean = apply(predictor, 2, function(x) mean(x[response == "AML"]))
    numerator = sum(response == "ALL")*(ALL_mean-overall)^2+sum(response == "AML")*(AML_mean-overall)^2
    denumerator = colSums((t(t(predictor[response == "ALL", ])-ALL_mean))^2)+colSums((t(t(predictor[response == "AML", ])-AML_mean))^2)
    numerator/denumerator
}

- **Nearest Neighbor**

In [4]:
k = seq(1, 21, 2)
# Distance for each test for NN classifiers
Distance = function(predictor, test){
    1- apply(predictor, 1, cor, test)
}
# NN classification process
nn = function(test, pk, learning, response){
     distance = Distance(learning, test)
     index = order(distance)[1:pk]
     cl = ifelse(sum(response[index] == "AML")>sum(response[index]=="ALL"), "AML", "ALL")
     cl
}
# leave-one-cross-validation to tune k
mycv= function(pk,learning,response){
    error = 0
    for(i in 1:nrow(learning)){
        cl = nn(learning[i,], pk, learning[-i, ], response[-i])
        error = error+(cl == response[i])
    }
    error
}
error_count = numeric(N)
for(i in 1:N){
    # split data
    nn_index = mysplit(nrow(scale_golub_merge))
    nn_train_p = scale_golub_merge[-nn_index,]
    nn_train_r = golub_merge_r[-nn_index]
    nn_test_p = scale_golub_merge[nn_index,]
    nn_test_r = golub_merge_r[nn_index]
    # gene selection
    temp_bw = order(BW(nn_train_p, nn_train_r), decreasing = T)[1:p]
    nn_train_p = nn_train_p[,temp_bw]
    nn_test_p = nn_test_p[,temp_bw]
    # cross-validation to choose k
    choose_k = sapply(k,mycv, learning = nn_train_p, response = nn_train_r)
    # nn classification
    nn_r = apply(nn_test_p,1, nn, k[which.min(choose_k)], nn_train_p, nn_train_r)
    error_count[i] = sum(nn_r != nn_test_r)
}
c(Median = median(error_count), Upper_quartile = quantile(error_count, 0.75))

Median Upper_quartile.75% 
                 1                  2

- **Desicion Tree(CART)**

This part is accomplished using tree() function in R. Since not much is mentioned in the paper about the set up of the decision tree, I make the assumption that the size of the tree is 3.

In [5]:
cbine_data = data.frame(response = factor(golub_merge_r), scale_golub_merge)
tree_mdl = tree(response~.,data = cbine_data)
cv_error = replicate(N, ceiling(cv.tree(tree_mdl, , prune.tree, K = 10, method = "misclass")$dev[1]/10))
c(Median = median(cv_error), Upper_quartile = quantile(cv_error, 0.75))

Median Upper_quartile.75% 
                 2                  2

- **Bagging**
Implemented without using packages in R.

In [6]:
bg_error = numeric(N)
my_baghelper = function(train, test){
    bg = sample(nrow(train), replace = T)
    temp_md = tree(response~., data = train[bg, ])
    predict(temp_md, test, type = "class")
}
for(i in 1:N){
    bg_index = mysplit(nrow(cbine_data))
    bg_train = cbine_data[-bg_index,]
    bg_test = cbine_data[bg_index,]
    
    # gene selection
    temp_bw = order(BW(bg_train[, -1], bg_train$response), decreasing = T)[1:p]
    bg_train_p = data.frame(response = bg_train$response, bg_train[,temp_bw+1])
    bg_test_p= data.frame(response = bg_test$response, bg_test[,temp_bw+1])
    dim(bg_train_p)
    t1 = replicate(B, my_baghelper(bg_train_p, bg_test_p))
    pred = apply(t1, 1, function(x) ifelse(sum(x == "AML")>sum(x =="ALL"), "AML", "ALL"))
    bg_error[i] = sum(pred != bg_test_p$response)
}
c(Median = median(bg_error), Upper_quartile = quantile(bg_error, 0.75))

Median Upper_quartile.75% 
                 2                  3

- **Bagging with CPD**


In [7]:
d = 0.75
CPD = function(d, x1, x2){
    a = runif(nrow(x1), 0, d)
    a*x1+(1-a)*x2
}
my_cpdhelper = function(train, test){
    id1 = sample(nrow(train), replace = T)
    id2 = sample(nrow(train), replace = T)
    temp = CPD(d, train[id1, -1], train[id2,-1])
    temp_md = tree(response~., data = data.frame(temp, response = train$response[id1]))
    predict(temp_md, test, type = "class")
}
cpd_error = numeric(N)
for(i in 1:N){
    cpd_index = mysplit(nrow(cbine_data))
    cpd_train = cbine_data[-cpd_index,]
    cpd_test = cbine_data[cpd_index,]
    
    # gene selection
    temp_bw = order(BW(cpd_train[, -1], cpd_train$response), decreasing = T)[1:p]
    cpd_train_t = data.frame(response = cpd_train$response, cpd_train[,temp_bw+1])
    cpd_test_t= data.frame(response = cpd_test$response, cpd_test[,temp_bw+1])
   
    t1 = replicate(B, my_cpdhelper(cpd_train_t, cpd_test_t))
    pred = apply(t1, 1, function(x) ifelse(sum(x == "AML")>sum(x =="ALL"), "AML", "ALL"))
    cpd_error[i] = sum(pred != cpd_test_t$response)
}
c(Median = median(cpd_error), Upper_quartile = quantile(cpd_error, 0.75))

Median Upper_quartile.75% 
                 2                  3

- **AdaBoost**
Using adaboost function from fastAdaboost package in R.

In [8]:
ada_error = numeric(N)
for(i in 1:N){
    ad_index = mysplit(nrow(cbine_data))
    ad_train = cbine_data[-ad_index,]
    ad_test = cbine_data[ad_index,]
    
    # gene selection
    temp_bw = order(BW(ad_train[, -1], ad_train$response), decreasing = T)[1:p]
    ad_train_t = data.frame(response = ad_train$response, ad_train[,temp_bw+1])
    ad_test_t= data.frame(response = ad_test$response, ad_test[,temp_bw+1])
    
    ada_cl = adaboost(response~., data = ad_train_t, 50)
    ada_test_pr = predict(ada_cl, ad_test_t)$class
   ada_error[i] = sum(ada_test_pr != ad_test_t$response)
}
c(Median = median(ada_error), Upper_quartile = quantile(ada_error, 0.75))

Median Upper_quartile.75% 
                 1                  2

- **FLDA**
As the current LDA is very similar to the FLDA, lda function in MASS package is used to accomplish this part.

In [9]:
flda_error = numeric(N)
for(i in 1:200){
    flda_index = mysplit(nrow(cbine_data))
    flda_train = cbine_data[-flda_index,]
    flda_test = cbine_data[flda_index,]
    
    # gene selection
    temp_bw = order(BW(flda_train[, -1], flda_train$response), decreasing = T)[1:p]
    flda_train_t = data.frame(response = flda_train$response, flda_train[,temp_bw+1])
    flda_test_t= data.frame(response = flda_test$response, flda_test[,temp_bw+1])
    
    flda_md = MASS::lda(response~., data = flda_train_t)
    flda_pred = predict(flda_md, flda_test_t)$class
    flda_error[i] = sum(flda_pred != flda_test_t$response)
}
c(Median = median(flda_error), Upper_quartile = quantile(flda_error, 0.75))

Median Upper_quartile.75% 
                 4                  5

- **DLDA**

Accomplished using R dlda in sparsediscrim packages.

In [10]:
dlda_error = numeric(N)
for(i in 1:200){
    dlda_index = mysplit(nrow(cbine_data))
    dlda_train = cbine_data[-dlda_index,]
    dlda_test = cbine_data[dlda_index,]
    
    # gene selection
    temp_bw = order(BW(dlda_train[, -1], dlda_train$response), decreasing = T)[1:p]
    dlda_train_t = data.frame(response = dlda_train$response, dlda_train[,temp_bw+1])
    dlda_test_t= data.frame(response = dlda_test$response, dlda_test[,temp_bw+1])
    
    dlda_md = dlda(response~., data = dlda_train_t)
    dlda_pred = predict(dlda_md, dlda_test_t[, -1])$class
    dlda_error[i] = sum(dlda_pred != dlda_test_t$response)
}
c(Median = median(dlda_error), Upper_quartile = quantile(dlda_error, 0.75))

Median Upper_quartile.75% 
                 1                  1

- **DQDA**

In [11]:
dqda_error = numeric(N)
for(i in 1:200){
    dqda_index = mysplit(nrow(cbine_data))
    dqda_train = cbine_data[-dqda_index,]
    dqda_test = cbine_data[dqda_index,]
    
    # gene selection
    temp_bw = order(BW(dqda_train[, -1], dqda_train$response), decreasing = T)[1:p]
    dqda_train_t = data.frame(response = dqda_train$response, dqda_train[,temp_bw+1])
    dqda_test_t= data.frame(response = dqda_test$response, dqda_test[,temp_bw+1])
    
    dqda_md = dlda(response~., data = dqda_train_t)
    dqda_pred = predict(dqda_md, dqda_test_t[,-1])$class
    dqda_error[i] = sum(dqda_pred != dqda_test_t$response)
}
c(Median = median(dqda_error), Upper_quartile = quantile(dqda_error, 0.75))

Median Upper_quartile.75% 
                 1                  1